In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
import folium
from folium import Map, FeatureGroup, Marker, LayerControl
from shapely.geometry import Point

import requests
import json
import glob
import time

from openrouteservice import client

In [2]:
tn_hosp_csv = pd.read_csv('../data/tn_hospitals.csv')

tn_hospitals = gpd.GeoDataFrame(tn_hosp_csv,
                        geometry = gpd.points_from_xy(tn_hosp_csv['billing_lng'], tn_hosp_csv['billing_lat']))

tn_hospitals

,billing_npi,billing_name,billing_class,billing_spec,billing_city,billing_county,billing_state,billing_zip,billing_lat,billing_lng,...,location_name,location_class,location_spec,location_city,location_county,location_state,location_zip,location_lat,location_lng,geometry
0,1558365890,METHODIST HEALTHCARE - MEMPHIS HOSPITALS,General Acute Care Hospital,NaN,MEMPHIS,Shelby County,TN,38104,35.137028,-90.018534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-90.01853 35.13703)
1,1144213117,REGIONAL ONE HEALTH,General Acute Care Hospital,NaN,MEMPHIS,Shelby County,TN,38103,35.142558,-90.031401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-90.03140 35.14256)
2,1093705428,JACKSON MADISON COUNTY GENERAL HOSPITAL,General Acute Care Hospital,NaN,JACKSON,Madison County,TN,38301,35.639689,-88.834424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-88.83442 35.63969)
3,1538164090,UNIVERSITY OF TENNESSEE MEDICAL CENTER,General Acute Care Hospital,NaN,KNOXVILLE,Knox County,TN,37920,35.935557,-83.948610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-83.94861 35.93556)
4,1861479545,MAURY REGIONAL MEDICAL CENTER,General Acute Care Hospital,NaN,COLUMBIA,Maury County,TN,38401,35.606430,-87.065251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.06525 35.60643)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,1942225727,INFECTIOUS DISEASE,General Acute Care Hospital,NaN,JACKSON,Madison County,TN,38301,35.639689,-88.834424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-88.83442 35.63969)
123,1104137215,RIVERVIEW REGIONAL MEDICAL CENTER,General Acute Care Hospital,Critical Access,CARTHAGE,Smith County,TN,37030,36.262289,-85.950536,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-85.95054 36.26229)
124,1558672543,TROUSDALE MEDICAL CENTER,General Acute Care Hospital,Critical Access,HARTSVILLE,Trousdale County,TN,37074,36.387383,-86.167723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-86.16772 36.38738)
125,1548315153,T C THOMPSON CHILDREN'S HOSPITAL,General Acute Care Hospital,NaN,CHATTANOOGA,Hamilton County,TN,37403,35.049787,-85.290737,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-85.29074 35.04979)


In [3]:
# importing API credentials from a file
with open('../data/openrouteservice_key.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['api_key']

In [4]:
tn_hospitals_short = tn_hospitals[['billing_name', 'billing_lat', 'billing_lng']].rename(columns = {'billing_name': 'hospital', 'billing_lat': 'lat', 'billing_lng': 'lng'})

In [5]:
tn_hospitals_short.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   hospital  127 non-null    object 
 1   lat       127 non-null    float64
 2   lng       127 non-null    float64
dtypes: float64(2), object(1)
memory usage: 3.1+ KB


In [6]:
proc_hosp = []

for hospital, lat, lng in tn_hospitals_short.values:
    
    if not hospital in proc_hosp:
        print(hospital)
        point = [lat, lng]

        params_iso = {
            'profile': 'driving-car',
            'range': [900, 1800], #interval in seconds 900 = 15 mins
            'attributes': ['total_pop'],
            'locations': [point[::-1]]
        }

        try:
            clnt = client.Client(key = api_key)
            r = clnt.isochrones(**params_iso)

            for feature in r['features']:
                feature['properties']['name'] = hospital
            
            with open (f'../data/isochrones/{hospital}.json', 'w') as f:
                f.write(json.dumps(r))
                proc_hosp.append(hospital)

            

        except:
            print(f'Problem processing {hospital}')

        time.sleep(2)

METHODIST HEALTHCARE - MEMPHIS HOSPITALS
REGIONAL ONE HEALTH
JACKSON MADISON COUNTY GENERAL HOSPITAL
UNIVERSITY OF TENNESSEE MEDICAL CENTER
MAURY REGIONAL MEDICAL CENTER
TRISTAR CENTENNIAL MEDICAL CENTER
ASCENSION SAINT THOMAS HOSPITAL
TRISTAR SOUTHERN HILLS MEDICAL CENTER
HENRY COUNTY MEDICAL CENTER
BAPTIST MEMORIAL HOSPITAL
ASCENSION SAINT THOMAS RUTHERFORD
NORTHCREST MEDICAL CENTER
ERLANGER MEDICAL CENTER
STARR REGIONAL MEDICAL CENTER
ST. FRANCIS HOSPITAL
SOUTHERN TENNESSEE REGIONAL HEALTH SYSTEM WINCHESTER
PENINSULA DIV PARKWEST MC
BRISTOL REGIONAL MEDICAL CENTER
SOUTHERN TENNESSEE REGIONAL HEALTH SYSTEM LAWRENCEBURG
VANDERBILT UNIVERSITY MEDICAL CENTER
MEMORIAL HOSPITAL
TRISTAR HORIZON MEDICAL CENTER
WEST TENNESSEE HEALTHCARE DYERSBURG HOSPITAL
PARKRIDGE EAST HOSPITAL
LAUDERDALE COMMUNITY HOSPITAL
WEST TENNESSEE HEALTHCARE VOLUNTEER HOSPITAL
BALLAD HEALTH MEDICAL ASSOCIATES
LINCOLN MEDICAL CENTER
MORRISTOWN HAMBLEN HEALTHCARE SYSTEM
FRANKLIN WOODS COMMUNITY HOSPITAL
BAPTIST MEMORI